In [59]:
from typing import Any, Optional, NamedTuple

from collections import defaultdict
from datetime import date
from pathlib import Path
import re

import py7zr

LOG_PATH = Path("C:/Users/zanfa/Documents/Star Wars - The Old Republic/CombatLogs")
ARCHIVES_PATH = LOG_PATH / "archive"
RE_LOG_NAME = re.compile(r"combat_(?P<year>[0-9]+)-(?P<month>[0-9]+)-(?P<day>[0-9]+)", re.I)
RE_ARCHIVE_NAME = re.compile(r"combat_(?P<year>[0-9]+)-(?P<month>[0-9]+)(?P<group>A|B)", re.I)

class GroupSpec(NamedTuple):
    year: int
    month: int
    half: str
    
    @classmethod
    def curent(cls) -> "GroupSpec":
        return cls.from_date(date.today())
    
    @classmethod
    def from_date(cls, d: date) -> "GroupSpec":
        return GroupSpec(
            d.year,
            d.month,
            "A" if d.day <= 15 else "B",
        )
    
    @classmethod
    def from_log_filename(cls, filename: str) -> Optional["GroupSpec"]:
        m = RE_LOG_NAME.match(filename)

        if not m:
            return None

        return cls(
            int(m['year']),
            int(m['month']),
            "A" if  int(m['day']) <= 15 else "B",
        )

    @classmethod
    def from_archive_filename(cls, filename: str) -> Optional["GroupSpec"]:
        m = RE_ARCHIVE_NAME.match(filename)

        if not m:
            return None

        return cls(
            int(m['year']),
            int(m['month']),
            m['group'].upper(),
        )
    
    @property
    def archive_filename(self) -> str:
        return f"combat_{self.year:04d}-{self.month:02d}{self.half.upper()}.7z"
    
    def __str__(self):
        return f"{self.year:04d}-{self.month:02d}-{self.half.upper()}"

class CombatLogs:
    def __init__(self, ):
        self.log_files: dict[GroupSpec, list[str]] = defaultdict(list)
        self.archives: dict[GroupSpec, str] = {}
        
    def log_group_exists(self, group_name: str) -> bool:
        return group_name in self.log_files
    
    def delete_log_group(self, group_name: str):
        del self.log_files[group_name]
        
    @classmethod
    def read_path(cls, path: Path):
        obj = cls()
        
        for file in path.iterdir():
            if not file.is_file():
                print(f"Skipping non-file {file.name}")

            elif file.suffix == ".txt":
                group = GroupSpec.from_log_filename(file.name)

                if group is None:
                    print(f"Unknown log file name format: {file.name}")
                    continue

                obj.log_files[group].append(file.name)
                # print(f"Grouped {file.name} in {group}")

            elif file.suffix == ".7z":
                group = GroupSpec.from_archive_filename(file.name)

                if group is None:
                    print(f"Unknown archive file name format: {file.name}")
                    continue

                if group in obj.archives:
                    print(f"Duplicate archive for group {group} ({file.name})")
                    continue

                obj.archives[group] = file.name
                # print(f"Found archive for {group} ({file.name})")

            else:
                print(f"Unknown file type: {file.name}")
        
        return obj
        
    
current_group = GroupSpec.curent()
cl = CombatLogs.read_path(LOG_PATH)

if cl.log_group_exists(current_group):
    # print(f"Ignoring current log files in: {current_group}")
    cl.delete_log_group(current_group)
        
for group in archives:
    if group in log_files:
        print(f"Found log files and archive for same group: {group}")
        del log_files[group]
        continue

if len(cl.log_files) <= 0:
    print("No log files to process")

for group, filenames in cl.log_files.items():
    filenames.sort()
    print(f"Archiving {len(filenames)} files in group {group} to {group.archive_filename}...")
    archive_path = ARCHIVES_PATH / group.archive_filename
    
    print(archive_path)
    
    if archive_path.exists():
        print(f"Archive file for {group} ({archive_path}) already exists...")
        continue
    
    compress_filter = [{'id': py7zr.FILTER_LZMA2, 'preset': py7zr.PRESET_DEFAULT}]

    with py7zr.SevenZipFile(archive_path, "w", filters=compress_filter) as archive_fh:
        f_count = len(filenames)
        f_dlen = len(str(f_count))
        
        print(f"Archiving {f_count:d} files... ", end="")
        for fx, filename in enumerate(filenames):
            file_path = LOG_PATH / filename
            
            archive_fh.write(file_path, arcname=file_path.name)
            print("#", end="")
                
        print(" Done!")
        
        print(f"Deleting source files...", end="")
        for fx, filename in enumerate(filenames):
            file_path = LOG_PATH / filename
            file_path.unlink()
                
        print(" Done!")

No log files to process
